In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data2014 = pd.read_csv("../Week7/Input/2014dataset.csv",low_memory=False)
data2015 = pd.read_csv("../Week7/Input/2015dataset.csv",low_memory=False)
data2016 = pd.read_csv("../Week7/Input/2016dataset.csv",low_memory=False)

In [3]:
important_fields = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME", "STATE", "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
                    "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)", 
                    "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                    "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                    "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                    "1.11FederalRiskCorridorsProgramnetpayments/(charges)2HealthInsuranceINDIVIDUALTotalasof3/31/15"]

data2014 = data2014[important_fields].dropna(how='all')
data2015 = data2015[important_fields].dropna(how='all')
data2016 = data2016[important_fields].dropna(how='all')

def add_columns(playground1):
    playground1["IndPremiumPMM"] = playground1["1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndCostsPMM"] = playground1["2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndRTPMM"] = playground1["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndReinsPMM"] = playground1["REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndRCPMM"] = playground1["1.11FederalRiskCorridorsProgramnetpayments/(charges)2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]  
    playground1["TotalPA"] = playground1["IndPremiumPMM"] - playground1["IndCostsPMM"]
    playground1["TotalPB"] = playground1["TotalPA"] + playground1["IndRTPMM"]
    playground1["TotalPC"] = playground1["TotalPB"] + playground1["IndReinsPMM"]
    playground1["TotalPD"] = playground1["TotalPB"] + playground1["IndRCPMM"]
    playground1["TotalPE"] = playground1["TotalPC"] + playground1["IndRCPMM"]

add_columns(data2014)
add_columns(data2015)
add_columns(data2016)

In [4]:
merged2 = data2014.merge(data2015, on='HIOS ID', how='outer', suffixes=('_2014', '_2015'))
merged2 = merged2.where(merged2["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014"] > 2000).dropna(how='all')
merged2 = merged2.where(merged2["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2015"] > 2000).dropna(how='all')
merged3 = merged2.merge(data2016, on='HIOS ID', how='outer', suffixes=('', '_2016'))
#merged3 = merged3.where(merged3["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"] > 2000).dropna(how='all')
merged3.to_csv("MarketExitAll_week10.csv")